In [ ]:
import os
import cv2
import numpy as np
import random
import shutil

# --- Cấu hình ---
data_dir = "dataset_clean"           # thư mục gốc chứa ảnh: forward_..., left_..., right_...
output_dir = "balanced_data"   # thư mục mới để save tất cả ảnh
target_per_class = 400         # số ảnh mong muốn cho left và right

# Tạo thư mục output, nếu đã tồn tại thì xóa rồi tạo mới
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)

# Copy tất cả ảnh gốc sang output_dir
for fn in os.listdir(data_dir):
    if fn.lower().endswith(".jpg"):
        shutil.copy(os.path.join(data_dir, fn),
                    os.path.join(output_dir, fn))

# Hàm augment (không flip)
def augment_image(img):
    h, w = img.shape[:2]
    augmented = []
    # 1) Xoay nhẹ
    for angle in [-15, -10, -5, 5, 10, 15]:
        M = cv2.getRotationMatrix2D((w/2, h/2), angle, 1.0)
        aug = cv2.warpAffine(img, M, (w, h), borderMode=cv2.BORDER_REPLICATE)
        augmented.append(aug)
    # 2) Dịch chuyển
    for _ in range(3):
        tx = random.randint(-20, 20)
        ty = random.randint(-20, 20)
        M = np.float32([[1, 0, tx], [0, 1, ty]])
        aug = cv2.warpAffine(img, M, (w, h), borderMode=cv2.BORDER_REPLICATE)
        augmented.append(aug)
    # 3) Thay đổi độ sáng
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    for delta in [30, -30]:
        hsv2 = hsv.copy()
        hsv2[:, :, 2] = np.clip(hsv2[:, :, 2] + delta, 0, 255)
        aug = cv2.cvtColor(hsv2, cv2.COLOR_HSV2BGR)
        augmented.append(aug)

    return augmented

# Hàm cân bằng 1 lớp
def balance_label(label):
    # 1) Liệt kê file hiện có trong output_dir
    files = [f for f in os.listdir(output_dir)
             if f.startswith(label + "_") and f.lower().endswith(".jpg")]
    count = len(files)
    print(f"Lớp {label}: đang có {count} ảnh, mục tiêu {target_per_class}")
    # 2) Xác định id để đặt tên file augment mới
    #    tìm số lớn nhất trong các file hiện tại
    existing_ids = []
    for f in files:
        parts = f.split("_")
        if len(parts) >= 2 and parts[-1].endswith(".jpg"):
            idx = parts[-1].split(".jpg")[0]
            if idx.isdigit():
                existing_ids.append(int(idx))
    next_id = max(existing_ids) + 1 if existing_ids else 0

    # 3) Bắt đầu augment cho đến đủ target
    while count < target_per_class:
        src_file = random.choice(files)
        img = cv2.imread(os.path.join(output_dir, src_file))
        for aug in augment_image(img):
            new_name = f"{label}_{next_id}.jpg"
            cv2.imwrite(os.path.join(output_dir, new_name), aug)
            next_id += 1
            count += 1
            if count >= target_per_class:
                break

    print(f"--> Hoàn thành: lớp {label} có {count} ảnh")

# Chạy cân bằng cho left và right
for lbl in ["left", "right"]:
    balance_label(lbl)

print("\nĐã cân bằng xong, tất cả ảnh nằm trong thư mục:", output_dir)


Lớp left: đang có 33 ảnh, mục tiêu 200
--> Hoàn thành: lớp left có 200 ảnh
Lớp right: đang có 57 ảnh, mục tiêu 200
--> Hoàn thành: lớp right có 200 ảnh

Đã cân bằng xong, tất cả ảnh nằm trong thư mục: balanced_data
